In [ ]:
%pip install -qqqU langchain-openai langgraph

In [ ]:
import os

from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_aws import BedrockEmbeddings
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

embeddings = BedrockEmbeddings()

qdrant_client = QdrantClient(
    location=os.getenv("VECTOR_STORE_URL", "http://localhost:6333"),
    api_key=os.getenv("VECTOR_STORE_API_KEY", "api_key"),
)

In [ ]:
regulamento_vector_store = QdrantVectorStore(
    client=qdrant_client, collection_name="regulamento_dos_cursos_de_graduacao_da_UFRN", embedding=embeddings
)

regulamento_retriever = regulamento_vector_store.as_retriever(search_kwargs={"k": 20})

In [ ]:
calendario_vector_store = QdrantVectorStore(
    client=qdrant_client, collection_name="calendario_universitario_2025", embedding=embeddings
)

calendario_retriever = calendario_vector_store.as_retriever(search_kwargs={"k": 50})

In [ ]:
from langchain.tools.retriever import create_retriever_tool

regulamentos_tool = create_retriever_tool(
    regulamento_retriever,
    "regulamentos_graducacao",
    "Artigos do Regulamento dos Cursos Regulares de Graduação"
    )

calendario_tool = create_retriever_tool(
    calendario_retriever,
    "calendario_2025",
    "Calendário Universitário da UFRN 2025"
    )

tools = [regulamentos_tool, calendario_tool]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt= "Você é o mascote da UFRN responsável por responder dúvidas sobre "
            "questões acadêmicas. Responda somente perguntas relacionadas a "
            "esse contexto.",
)

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [ ]:
inputs = {
    "messages": [
        ("user", "Como cancelar meu curso?")
    ]
}

print_stream(agent.stream(inputs, stream_mode="values"))

In [ ]:
inputs = {
    "messages": [
        ("user", "Quando começam as aulas?")
    ]
}
print_stream(agent.stream(inputs, stream_mode="values"))

In [ ]:
inputs = {
    "messages": [
        ("user", "Quem é chico mota?")
    ]
}
print_stream(agent.stream(inputs, stream_mode="values"))

In [ ]:
inputs = {
    "messages": [
        ("user", "Ola?")
    ]
}
print_stream(agent.stream(inputs, stream_mode="values"))

In [ ]:
inputs = {
    "messages": [
        ("user", "Você tem conhecimento sobre o que?")
    ]
}
print_stream(agent.stream(inputs, stream_mode="values"))